#Initialization

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window


# Business Logic

In [0]:
query = """
SELECT
    ROW_NUMBER() OVER (ORDER BY ci.customer_id) AS customer_key,
    ci.customer_id,
    ci.customer_number,
    ci.first_name,
    ci.last_name,
    la.country,
    ci.marital_status,
    CASE
        WHEN ci.gender <> 'n/a' THEN ci.gender
        ELSE COALESCE(ca.gender, 'n/a')
    END AS gender,
    ca.birth_date AS birthdate,
    ci.created_date AS create_date
FROM silver.crm_cust_info ci
LEFT JOIN silver.erp_cust_az12 ca
    ON ci.customer_number = ca.customer_id
LEFT JOIN silver.erp_loc_a101 la
    ON ci.customer_number = la.company_id
"""
df = spark.sql(query)


## Renaming

# Writing Gold Table

In [0]:
(
    df.write
      .mode("overwrite")
      .format("delta")
      .saveAsTable("gold.dim_customers")
)


In [0]:
%sql
SELECT *
FROM workspace.silver.dim_customers